### 开始

在本教程中，我们将学习如何在 LangChain 中创建简单的链。我们将学习如何创建链、向其添加组件并运行它。
在本教程中，我们将介绍：

- 使用简单的 LLM 链
- 创建顺序链
- 创建自定义链

### 为什么我们需要链条？

链允许我们将多个组件组合在一起以创建一个单一的、连贯的应用程序。例如，我们可以创建一个接受用户输入的链，使用 PromptTemplate 对其进行格式化，然后将格式化后的响应传递给 LLM。我们可以通过将多个链组合在一起，或者通过将链与其他组件组合来构建更复杂的链。

### 快速开始：使用`LLMChain`

这`LLMChain`是一个简单的链，它接受一个提示模板，用用户输入格式化它并返回来自 LLM 的响应。
要使用`LLMChain`，首先创建一个提示模板。


In [2]:

import os
import sys               
sys.path.append(os.path.dirname(os.path.abspath(".."))) 
from API_KEYS import InitAzureEnv,AzureDeploymentName

InitAzureEnv()

azure


In [3]:
from langchain.prompts import PromptTemplate
from langchain.llms import AzureOpenAI
from langchain.chains import LLMChain

llm = AzureOpenAI(
    temperature=0,
    deployment_name = AzureDeploymentName.LLM
)
prompt = PromptTemplate(
    input_variables=["product"],
    template="推荐一个在中国专业做 {product}产品的公司?",
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


我们现在可以创建一个非常简单的链，它将接受用户输入，用它格式化提示，然后将它发送给 LLM。


In [4]:
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("手机"))



深圳市联想科技有限公司是一家专业从事手机产品的公司，拥有丰富的研发经验，拥有多款热门手机产品，并且在中国市场有着良好的口碑。


如果有多个变量，您可以使用字典一次输入它们。


In [8]:
prompt = PromptTemplate(
    input_variables=["country", "product"],
    template="推荐一个在{country}专业做 {product}产品的公司?",
)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'country': "美国",
    'product': "手机"
 }))



Apple、Motorola、HTC和Samsung等电话制造商。


您也可以使用`LLMChain`聊天模型：


In [9]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["country", "product"],
            template="推荐一个在{country}专业做 {product}产品的公司?",
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = AzureChatOpenAI(
    temperature=0,
    deployment_name = AzureDeploymentName.CHAT
)
chain = LLMChain(llm=chat, prompt=chat_prompt_template)
print(chain.run({
    'country': "美国",
    'product': "手机"
 }))

苹果公司（Apple Inc.）是一个在美国专业做手机产品的公司，其旗下的iPhone系列手机在全球范围内广受欢迎。


#### 调用链的不同方式

所有继承自的类都`Chain`提供了几种运行链逻辑的方法。最直接的一种是使用`__call__`:


In [10]:
chat = AzureChatOpenAI(
    temperature=0,
    deployment_name = AzureDeploymentName.CHAT
)
prompt_template = "推荐一个在中国专业做 {product}产品的公司?"
llm_chain = LLMChain(
    llm=chat,
    prompt=PromptTemplate.from_template(prompt_template)
)

llm_chain(inputs={"product":"手机"})

{'product': '手机',
 'text': '华为技术有限公司是一家在中国专业做手机产品的公司，它是全球领先的信息和通信技术（ICT）解决方案供应商之一，拥有自主研发的芯片、操作系统和通信技术，其手机产品在国内外市场上都有很高的知名度和市场占有率。'}

默认情况下，`__call__`返回输入和输出键值。`return_only_outputs`您可以通过设置为将其配置为仅返回输出键值`True`。


In [12]:
llm_chain({"product":"手机"},return_only_outputs=True)

{'text': '华为技术有限公司是一家在中国专业做手机产品的公司，它是全球领先的信息和通信技术（ICT）解决方案供应商之一，拥有自主研发的芯片、操作系统和通信技术，其手机产品在国内外市场上都有很高的知名度和市场占有率。'}

如果`Chain`只输出一个输出键（即只有一个元素在它的 output_keys），你可以使用 run 方法。
请注意，run 输出的是字符串而不是字典。


In [13]:
# llm_chain only has one output key, so we can use run
llm_chain.output_keys

['text']

提示：您可以通过其方法轻松地将`Chain`对象集成为`Tool`您的对象。请在此处查看示例。`Agentrun`


#### 向链中添加内存

`Chain`支持将`BaseMemory`对象作为`memory`参数，允许`Chain`对象在多次调用中持久保存数据。换句话说，它创建了`Chain`一个有状态的对象。


In [14]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

conversation.run("推荐一个在中国专业做手机产品的公司?")
conversation.run("再推荐4家?")

'除了之前提到的华为、小米、OPPO和vivo之外，还有一些其他的手机公司也值得关注。例如，荣耀、realme、一加、联想等等。这些公司都有自己的特色和优势，您可以根据自己的需求和喜好选择其中一个。'

本质上，`BaseMemory`定义了一个如何`langchain`存储内存的接口。它允许通过方法读取存储的数据`load_memory_variables`并通过`save_context`方法存储新数据。


#### 调试链

`Chain`设置`verbose`为将在运行时`True`打印出`Chain`对象的一些内部状态。


In [15]:
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory(),
    verbose=True
)
conversation.run("推荐一个在中国专业做手机产品的公司?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 推荐一个在中国专业做手机产品的公司?
AI:

> Finished chain.


'中国有很多专业做手机产品的公司，其中一些知名的公司包括华为、小米、OPPO、vivo、魅族等等。这些公司都在中国市场拥有很高的知名度和市场份额。您可以根据自己的需求和喜好选择其中一个公司。'

### 组合链 SequentialChain

调用语言模型后的下一步是对语言模型进行一系列调用。我们可以使用顺序链来做到这一点，顺序链是按预定义顺序执行其链接的链。具体来说，我们将使用`SimpleSequentialChain`. 这是最简单的顺序链类型，其中每个步骤都有一个输入/输出，一个步骤的输出是下一个步骤的输入。

在本教程中，我们的顺序链将：<br>
首先，为产品创建公司名称。我们将重用`LLMChain`我们之前初始化的来创建这个公司名称。<br>
然后，为产品创建一个标语。我们将初始化一个新的`LLMChain`来创建这个标语，如下所示。


In [18]:
one_prompt = PromptTemplate(
    input_variables=["sport"],
    template="英国最出色的{sport}俱乐部是什么?",
)
chain_one = LLMChain(llm=llm, prompt=one_prompt)

second_prompt = PromptTemplate(
    input_variables=["club"],
    template="这家{club}历史上都有哪些著名运动员？",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

现在我们可以将这两个 LLMChain 结合起来，这样我们就可以一步创建一个公司名称和一个标语。


In [21]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

catchphrase = overall_chain.run("足球")
print(catchphrase)



> Entering new SimpleSequentialChain chain...


英国最出色的足球俱乐部是曼联队。曼联队是英格兰足球超级联赛的冠军，也是欧洲冠军联赛的冠军，拥有20个英格兰足球超级联赛冠军，12个足总杯冠军，5个欧洲冠军联赛冠军，3个欧洲联盟杯冠军，1个欧洲超级杯冠军，以及1个世界俱乐部冠军。
曼联队有许多著名的运动员，其中包括：贝克汉姆，罗纳尔多，埃里克森，卡里姆，卡洛斯，费尔南多，费尔南德斯，贝尔，卡里克，卡瓦尼，费尔纳多，卡拉斯科，罗伯特，马塞洛，罗伊斯，罗伊斯，罗伊斯，罗伊斯，罗伊斯，罗伊

> Finished chain.
曼联队有许多著名的运动员，其中包括：贝克汉姆，罗纳尔多，埃里克森，卡里姆，卡洛斯，费尔南多，费尔南德斯，贝尔，卡里克，卡瓦尼，费尔纳多，卡拉斯科，罗伯特，马塞洛，罗伊斯，罗伊斯，罗伊斯，罗伊斯，罗伊斯，罗伊


### `Chain`使用类创建自定义链

LangChain 提供了许多开箱即用的链，但有时您可能希望为您的特定用例创建自定义链。

为了创建自定义链,从类的子类开始 Chain，<br>
填写 input_keys 和 output_keys 属性，<br>
添加\_call 显示如何执行链的方法。<br>


In [14]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

现在，我们可以尝试运行我们调用的链。


In [15]:
prompt_1 = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["product"],
    template="What is a good slogan for a company that makes {product}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("colorful socks")
print(f"Concatenated output:\n{concat_output}")

Concatenated output:


Funky Footwear Company

"Brighten Up Your Day with Our Colorful Socks!"


That's it! For more details about how to do cool things with Chains, check out the [how-to guide](how_to_guides.rst) for chains.
